In [1]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import json

In [2]:
import re
import rapidfuzz

In [3]:
newoag=pd.read_json('oag_qa_20230512.json',lines=True)

In [4]:
newoag

,_id,qid,score,url,title,body,tags,answers
0,{'$oid': '645a083eeae567d13a86162e'},4834,6,https://patents.stackexchange.com/questions/48...,Cloud Computing - Suggesting customers migrate...,<p><strong>AN OVERBROAD PATENT ON suggesting c...,"[prior-art-request, pre-grant, ibm, cloud-stor...","[{'aid': '4956', 'qid': '4834', 'is_accepted':..."
1,{'$oid': '645a0840eae567d13a86162f'},13157,6,https://patents.stackexchange.com/questions/13...,Requesting prior art on Google machine learnin...,<p>Google is attempting to patent well known c...,"[prior-art-request, google, wo2014105866a1, us...","[{'aid': '13206', 'qid': '13157', 'is_accepted..."
2,{'$oid': '645a0842eae567d13a861630'},12028,4,https://patents.stackexchange.com/questions/12...,EDA software (IC design),<p>Looking for prior art on this:</p>\n\n<p><a...,"[prior-art-request, pre-grant, us20130290834, ...","[{'aid': '21228', 'qid': '12028', 'is_accepted..."
3,{'$oid': '645a0842c73f28c9898f4260'},110634,84,https://softwareengineering.stackexchange.com/...,Why would it ever be possible for Java to be f...,<p>Sometimes Java outperforms C++ in benchmark...,"[java, c++, performance]","[{'aid': '430340', 'qid': '110634', 'is_accept..."
4,{'$oid': '645a0842c73f28c9898f4261'},322714,76,https://softwareengineering.stackexchange.com/...,Are stacks the only reasonable way to structur...,<p>Most architectures I've seen rely on a call...,[computer-architecture],"[{'aid': '322732', 'qid': '322714', 'is_accept..."
...,...,...,...,...,...,...,...,...
19767,{'$oid': '645b81c7ec92098bbf259229'},41964,6,https://astronomy.stackexchange.com/questions/...,Is there a way to calculate the mass of of a s...,<p>Assume there is a star in a completely empt...,"[star, mass, stellar-remnants]","[{'aid': '42037', 'qid': '41964', 'is_accepted..."
19768,{'$oid': '645b81c9cfd7fc2d2dafb76b'},363226,76,https://mathoverflow.net/questions/363226/each...,Each mathematician has only a few tricks,"<p>The question &quot;<a href=""https://mathove...","[big-list, big-picture, gm.general-mathematics]","[{'aid': '363289', 'qid': '363226', 'is_accept..."
19769,{'$oid': '645b81d0cfd7fc2d2dafb76c'},115112,73,https://mathoverflow.net/questions/115112/how-...,How does &quot;modern&quot; number theory cont...,<p>I hope this question is appropriate for MO....,"[nt.number-theory, big-picture]","[{'aid': '115148', 'qid': '115112', 'is_accept..."
19770,{'$oid': '645b81d7ec92098bbf25922a'},41742,6,https://astronomy.stackexchange.com/questions/...,Would it be possible to discover an asteroid b...,<p>With existing or planned technology (e.g. t...,"[observational-astronomy, asteroids, planetary...","[{'aid': '41743', 'qid': '41742', 'is_accepted..."


In [5]:
newoag_title=newoag[['title','answers']]

In [6]:
newoag_title

,title,answers
0,Cloud Computing - Suggesting customers migrate...,"[{'aid': '4956', 'qid': '4834', 'is_accepted':..."
1,Requesting prior art on Google machine learnin...,"[{'aid': '13206', 'qid': '13157', 'is_accepted..."
2,EDA software (IC design),"[{'aid': '21228', 'qid': '12028', 'is_accepted..."
3,Why would it ever be possible for Java to be f...,"[{'aid': '430340', 'qid': '110634', 'is_accept..."
4,Are stacks the only reasonable way to structur...,"[{'aid': '322732', 'qid': '322714', 'is_accept..."
...,...,...
19767,Is there a way to calculate the mass of of a s...,"[{'aid': '42037', 'qid': '41964', 'is_accepted..."
19768,Each mathematician has only a few tricks,"[{'aid': '363289', 'qid': '363226', 'is_accept..."
19769,How does &quot;modern&quot; number theory cont...,"[{'aid': '115148', 'qid': '115112', 'is_accept..."
19770,Would it be possible to discover an asteroid b...,"[{'aid': '41743', 'qid': '41742', 'is_accepted..."


In [7]:
newoag_title_dict=newoag_title.to_dict()

In [8]:
import os
import glob

folder_path = "/Users/judyzhu/Desktop/2023 summer/oagqa-topic-v2"  # Replace with the actual folder path
files = glob.glob(os.path.join(folder_path, "*train*"))
merged_data=[]
for file in files: 
    with open(file, 'r') as file:
        json_data = json.load(file)
        df=pd.DataFrame(json_data)
        merged_data.append(df)
merged_data
joined_data=pd.concat(merged_data)

In [9]:
tsvfiles = glob.glob(os.path.join(folder_path, "*questions.tsv*"))
#data=pandas.read_csv('filename.tsv',sep='\t')
merged_data=[]
for file in tsvfiles: 
    df=pd.read_csv(file,sep='\t',names=['question','answer'])
    merged_data.append(df)
joined_data=pd.concat(merged_data)

In [10]:
Str_A = ['FuzzyWuzzy is a lifesaver!','is fuzzy wuzzy a life saver ']
Str_B = 'fuzzy wuzzy is a LIFE SAVER.' 
ratio = fuzz.ratio(Str_A, Str_B)
print('Similarity score: {}'.format(ratio))


Similarity score: 36


In [11]:
ratio

36

In [12]:
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [13]:
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.ratio(name, x)
        if score > min_score:
            max_name = x
            max_score = score
    return (max_name, max_score)

In [14]:

def match_names(name, list_names, min_score=0):
    matching_names = []
    for x in list_names:
        score = fuzz.ratio(name, x)
        if score > min_score:
            if score > 90:  # Check if the score is greater than 90
                matching_names.append((x, score))
    return matching_names

In [15]:

def match_names(name, list_names, min_score=0):
    matching_names = []
    for x in list_names:
        score = fuzz.ratio(name, x)
        if score > 90:  # Check if the score is greater than 90
            matching_names.append((x, score))
    return matching_names

In [16]:
#match_names(oldquestion, newquestion, min_score=0)

In [17]:
joined_data_name=joined_data[['question','answer']]
joined_data_name

,question,answer
0,Example of a quantum-mechanical theory with no...,"['toeplitz quantization of k\\""ahler manifolds..."
1,How can transverse waves on a string carry lon...,['The missing wave momentum mystery']
2,How do contact transformations differ from can...,['contact hamiltonian mechanics']
3,Reference for a type of &quot;multi-hamiltonia...,['Classical and quantum Nambu mechanics']
4,Issues with Newton&#39;s third law and Euler&#...,"[""mansuripur's paradox""]"
...,...,...
77,What do blind people dream when they dream?,['Visual imagery without visual perception?']
78,What is the cognitive neural mechanism of atte...,['Attention Induced Gain Stabilization in Broa...
79,"How to evaluate the study of ""only 39 of the 1...",['Statistics: <i>P</i> values are just the tip...
80,What do you think of the findings published in...,['Human Hippocampal Neurogenesis Persists thro...


In [18]:
joined_data_namedict=joined_data['question'].to_dict()

In [19]:
#newoag_title_dict
#joined_data_namedict

In [20]:
#newoag_title
#joined_data_name

In [21]:
#newoag_title
#joined_data_name

In [22]:
oldquestion=list(joined_data['question'])
newquestion=list(newoag['title'])

In [29]:
#dictionary: for each title, find all indexes that contains the same words
wordtoquestion={}
for title in newquestion:
    titlewords=set(title.lower().split())
    for question in oldquestion:
        questionlist=[]
        nonelist=[]
        questionwords=set(question.lower().split())
        if len(titlewords.intersection(questionwords))!=0:
            questionlist.append(question)
        else:
            nonelist.append(title)
        wordtoquestion[title]=questionlist
wordtoquestion

KeyboardInterrupt: 

In [ ]:
newoag_title["title"] = newoag_title["title"].apply(
  lambda x: process.extractOne(x, joined_data_name["question"], scorer=fuzz.partial_ratio)[0]
)

# Attempt to join the two dataframe
data = pd.merge(newoag_title, joined_data_name, on="country", how="left")
print(data.head())


In [ ]:
#no merging

In [ ]:
a={'apple', 'banana', 'cherry'}

b={'banana'}

In [ ]:
len(a.intersection(b))

In [25]:
def get_unique_words(series):
    unique_words = set()
    for text in series:
        words = text.split()
        unique_words.update(words)
    return unique_words

In [26]:
from fuzzywuzzy import fuzz, process
import pandas as pd

def find_non_matching_titles(titles, questions, min_score=90):
    non_matching_titles = []
    commonthreshold=0
    questions_words = get_unique_words(questions)
    for title in titles:
        matched = False
        title_words = set(title.lower().split())
        for question in questions:
            if len(title_words.intersection(questions_words))==0:
                non_matching_titles.append(title)
            else: 
                score = rapidfuzz.fuzz.ratio(title, question)
                if score > min_score:
                    matched = True
                    break
                if not matched:
                    non_matching_titles.append(title)
    return non_matching_titles



In [ ]:
# Find non-matching titles in newoag_title["title"]
non_matching_titles = find_non_matching_titles(newoag_title["title"], joined_data_name["question"])

print(non_matching_titles)

In [ ]:
oldquestion=list(joined_data['question'])
newquestion=list(newoag['title'])

In [30]:
def generate_inverted_index(list1, list2):
    inverted_index = {}
    for index, name in enumerate(list1):
        names = name.split()
        
        for n in names:
            if n in inverted_index:
                inverted_index[n].append((index+1, "List 1"))
            else:
                inverted_index[n] = [(index+1, "List 1")]
    
    # Process List 2
    for index, name in enumerate(list2):
        # Split the name into individual words
        names = name.split()
        
        # Iterate over each word in the name
        for n in names:
            if n in inverted_index:
                inverted_index[n].append((index+1, "List 2"))
            else:
                inverted_index[n] = [(index+1, "List 2")]
    
    return inverted_index

def find_names_in_single_list_with_index(inverted_index):
    single_list_names = {}
    
    for name, occurrences in inverted_index.items():
        if len(occurrences) == 1:
            list_id = occurrences[0][1]  # Get the list identifier
            index = occurrences[0][0]  # Get the index
            if list_id in single_list_names:
                single_list_names[list_id].append((name, index))
            else:
                single_list_names[list_id] = [(name, index)]
    
    return single_list_names
def find_names_in_both_lists_with_index(inverted_index):
    common_names = {}
    
    for name, occurrences in inverted_index.items():
        if len(occurrences) > 1:
            for occurrence in occurrences:
                list_id = occurrence[1]  # Get the list identifier
                index = occurrence[0]  # Get the index
                if list_id in common_names:
                    common_names[list_id].append((name, index))
                else:
                    common_names[list_id] = [(name, index)]
    
    return common_names

In [31]:
find_names_in_both_lists_with_index(generate_inverted_index(oldquestion, newquestion))

{'List 1': [('Example', 1),
  ('Example', 1726),
  ('Example', 1967),
  ('Example', 4183),
  ('Example', 7023),
  ('Example', 8108),
  ('Example', 8414),
  ('Example', 8543),
  ('Example', 8657),
  ('Example', 9163),
  ('Example', 9868),
  ('Example', 9877),
  ('Example', 10651),
  ('Example', 10700),
  ('Example', 12025),
  ('Example', 12366),
  ('Example', 13756),
  ('Example', 15154),
  ('Example', 15364),
  ('Example', 15400),
  ('of', 1),
  ('of', 4),
  ('of', 5),
  ('of', 8),
  ('of', 9),
  ('of', 17),
  ('of', 25),
  ('of', 27),
  ('of', 27),
  ('of', 27),
  ('of', 31),
  ('of', 32),
  ('of', 33),
  ('of', 33),
  ('of', 35),
  ('of', 40),
  ('of', 43),
  ('of', 48),
  ('of', 51),
  ('of', 52),
  ('of', 52),
  ('of', 57),
  ('of', 58),
  ('of', 60),
  ('of', 61),
  ('of', 64),
  ('of', 65),
  ('of', 68),
  ('of', 69),
  ('of', 73),
  ('of', 87),
  ('of', 91),
  ('of', 98),
  ('of', 98),
  ('of', 112),
  ('of', 113),
  ('of', 114),
  ('of', 129),
  ('of', 150),
  ('of', 151),
  ('

In [32]:
generate_inverted_index(oldquestion, newquestion)

{'Example': [(1, 'List 1'),
  (1726, 'List 1'),
  (1967, 'List 1'),
  (4183, 'List 1'),
  (7023, 'List 1'),
  (8108, 'List 1'),
  (8414, 'List 1'),
  (8543, 'List 1'),
  (8657, 'List 1'),
  (9163, 'List 1'),
  (9868, 'List 1'),
  (9877, 'List 1'),
  (10651, 'List 1'),
  (10700, 'List 1'),
  (12025, 'List 1'),
  (12366, 'List 1'),
  (13756, 'List 1'),
  (15154, 'List 1'),
  (15364, 'List 1'),
  (15400, 'List 1'),
  (1684, 'List 2'),
  (2842, 'List 2'),
  (5805, 'List 2'),
  (6370, 'List 2'),
  (7127, 'List 2'),
  (7413, 'List 2'),
  (7820, 'List 2'),
  (8264, 'List 2'),
  (9302, 'List 2'),
  (9928, 'List 2'),
  (12749, 'List 2'),
  (13904, 'List 2'),
  (14538, 'List 2'),
  (15095, 'List 2'),
  (15921, 'List 2'),
  (16224, 'List 2'),
  (16339, 'List 2'),
  (17222, 'List 2')],
 'of': [(1, 'List 1'),
  (4, 'List 1'),
  (5, 'List 1'),
  (8, 'List 1'),
  (9, 'List 1'),
  (17, 'List 1'),
  (25, 'List 1'),
  (27, 'List 1'),
  (27, 'List 1'),
  (27, 'List 1'),
  (31, 'List 1'),
  (32, 'List 1')

In [ ]:
len(find_names_in_single_list_with_index(generate_inverted_index(oldquestion, newquestion))['List 2'])

class InvertedIndex:
    """
    Inverted Index class.
    """
    def __init__(self, db):
        self.index = dict()
        self.db = db
    def __repr__(self):
        """
        String representation of the Database object
        """
        return str(self.index)
        
    def index_document(self, document):
        """
        Process a given document, save it to the DB and update the index.
        """
        
        # Remove punctuation from the text.
        clean_text = re.sub(r'[^\w\s]','', document['text'])
        terms = clean_text.split(' ')
        appearances_dict = dict()
        # Dictionary with each term and the frequency it appears in the text.
        for term in terms:
            term_frequency = appearances_dict[term].frequency if term in appearances_dict else 0
            appearances_dict[term] = Appearance(document['id'], term_frequency + 1)
            
        # Update the inverted index
        update_dict = { key: [appearance]
                       if key not in self.index
                       else self.index[key] + [appearance]
                       for (key, appearance) in appearances_dict.items() }
        self.index.update(update_dict)
        # Add the document into the database
        self.db.add(document)
        return document
    
    def lookup_query(self, query):
        """
        Returns the dictionary of terms with their correspondent Appearances. 
        This is a very naive search since it will just split the terms and show
        the documents where they appear.
        """
        return { term: self.index[term] for term in query.split(' ') if term in self.index }

appearance=InvertedIndex(newoag_title_dict).index_document(newoag_title_dict)

def highlight_term(id, term, text):
    replaced_text = text.replace(term, "\033[1;32;40m {term} \033[0;0m".format(term=term))
    return "--- document {id}: {replaced}".format(id=id, replaced=replaced_text)
def main():
    db = Database()
    index = InvertedIndex(db)
    document1 = {
        'id': '1',
        'text': 'The big sharks of Belgium drink beer.'
    }
    document2 = {
        'id': '2',
        'text': 'Belgium has great beer. They drink beer all the time.'
    }
    index.index_document(document1)
    index.index_document(document2)
    
    
    search_term = input("Enter term(s) to search: ")
    result = index.lookup_query(search_term)
    
    for term in result.keys():
        for appearance in result[term]:
            # Belgium: { docId: 1, frequency: 1}
            document = db.get(appearance.docId)
            print(highlight_term(appearance.docId, term, document['text']))
        print("-----------------------------")    
    
main()

In [1]:
#fuzzycouple
import re
text = 'Supporting "Temporal \'Reasoning by Mapping #@{ Calendar Expressions to Minimal\n Periodic Sets'
REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
text = re.sub(' +', ' ', text)
text = text.lower()
print(text)


supporting temporal reasoning by mapping calendar expressions to minimal periodic sets


In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/judyzhu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
STOPWORDS = stopwords.words('english')
STOPWORDS = set(STOPWORDS)
    
def text_prepare(text, STOPWORDS):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()

    # delete stopwords from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) 
    text = text.strip()
    return text

In [8]:
#find only the one that matches
def closest(title, questionlist):
    for question in questionlist:
        score=rapidfuzz.fuzz.ratio(title, question)
        
        
        
        
        
        
def match_names(name, list_names, min_score=0):
    matching_names = []
    for x in list_names:
        score = rapidfuzz.fuzz.ratio(title, question)
        if score > 90:  # Check if the score is greater than 90
            matching_names.append((x, score))
    return matching_names

IndentationError: expected an indented block (212158666.py, line 4)

In [ ]:
from fuzzywuzzy import fuzz, process
import pandas as pd

def find_non_matching_titles(titles, questions, min_score=90):
    non_matching_titles = []
    commonthreshold=0
    questions_words = get_unique_words(questions)
    for title in titles:
        matched = False
        title_words = set(title.lower().split())
        for question in questions:
            if len(title_words.intersection(questions_words))==0:
                non_matching_titles.append(title)
            else: 
                score = rapidfuzz.fuzz.ratio(title, question)
                if score > min_score:
                    matched = True
                    break
                if not matched:
                    non_matching_titles.append(title)
    return non_matching_titles

